In [5]:
import os
import pandas as pd
from datetime import datetime
from dbfread import DBF

# --- PARÂMETROS GLOBAIS (O que sua chefe pediu) ---
DATA_INICIO = '2026-01-04'
DATA_FIM    = '2026-01-31'

# Estrutura de Pastas
PASTA_INPUT  = '../01_Entrada'
PASTA_OUTPUT = '../02_Saida'
PASTA_DEMO   = '../data_demo'

os.makedirs(PASTA_OUTPUT, exist_ok=True)
os.makedirs(PASTA_DEMO, exist_ok=True)

def carregar_banco(caminho):
    ext = os.path.splitext(caminho)[1].lower()
    if ext == '.dbf':
        dbf = DBF(caminho, encoding='latin1', char_decode_errors='ignore')
        return pd.DataFrame(iter(dbf))
    elif ext == '.csv':
        return pd.read_csv(caminho, sep=';', encoding='latin1', low_memory=False)
    elif ext == '.xlsx':
        return pd.read_excel(caminho)
    return None

print(f"✅ Configuração pronta para o intervalo: {DATA_INICIO} a {DATA_FIM}")

✅ Configuração pronta para o intervalo: 2026-01-04 a 2026-01-31


In [6]:
print("--- Processando bases do GAL ---")
arquivos = os.listdir(PASTA_INPUT)
files_gal = [f for f in arquivos if ('IGM' in f.upper() or 'NS1' in f.upper()) and f.endswith('.csv')]

dfs_gal = []
for f in files_gal:
    df_t = carregar_banco(os.path.join(PASTA_INPUT, f))
    if df_t is not None:
        # Colunas requisitadas
        cols = ['Data do 1º Sintomas', 'IBGE Município Solicitante', 'IBGE Município de Residência']
        dfs_gal.append(df_t[[c for c in cols if c in df_t.columns]])

df_gal = pd.concat(dfs_gal, ignore_index=True)
df_gal['Data do 1º Sintomas'] = pd.to_datetime(df_gal['Data do 1º Sintomas'], errors='coerce', dayfirst=True)

# Filtro de Data
df_gal = df_gal[(df_gal['Data do 1º Sintomas'] >= DATA_INICIO) & (df_gal['Data do 1º Sintomas'] <= DATA_FIM)]

# Contagens por Solicitante e por Residência
resumo_gal_solic = df_gal.groupby('IBGE Município Solicitante').size().reset_index(name='Qtd_GAL_Solicitante')
resumo_gal_resid = df_gal.groupby('IBGE Município de Residência').size().reset_index(name='Qtd_GAL_Residencia')

# Padronização de códigos IBGE (6 dígitos)
resumo_gal_solic['COD_IBGE'] = resumo_gal_solic['IBGE Município Solicitante'].astype(str).str.replace(r'\.0$', '', regex=True)
resumo_gal_resid['COD_IBGE'] = resumo_gal_resid['IBGE Município de Residência'].astype(str).str.replace(r'\.0$', '', regex=True)

print(f"✅ GAL processado: {len(df_gal)} registros encontrados no período.")

--- Processando bases do GAL ---
✅ GAL processado: 893 registros encontrados no período.


In [7]:
print("--- Processando base do SINAN ---")
file_sinan = next((f for f in arquivos if 'DENGON2026' in f.upper() and f.endswith('.dbf')), None)

if file_sinan:
    df_sinan = carregar_banco(os.path.join(PASTA_INPUT, file_sinan))
    # ID_MUNICIP = Notificador
    df_sinan = df_sinan[['ID_MUNICIP', 'DT_SIN_PRI']]
    df_sinan['DT_SIN_PRI'] = pd.to_datetime(df_sinan['DT_SIN_PRI'], errors='coerce')
    
    # Filtro de Data
    df_sinan = df_sinan[(df_sinan['DT_SIN_PRI'] >= DATA_INICIO) & (df_sinan['DT_SIN_PRI'] <= DATA_FIM)]
    
    resumo_sinan = df_sinan.groupby('ID_MUNICIP').size().reset_index(name='Qtd_SINAN_Notificador')
    resumo_sinan['COD_IBGE'] = resumo_sinan['ID_MUNICIP'].astype(str).str.replace(r'\.0$', '', regex=True)
    
    print(f"✅ SINAN processado: {len(df_sinan)} notificações.")

--- Processando base do SINAN ---
✅ SINAN processado: 3110 notificações.


In [8]:
# Cruzamento com o Dicionário de Municípios
file_mun = next((f for f in arquivos if 'MUNIC' in f.upper()), None)
df_mun = carregar_banco(os.path.join(PASTA_INPUT, file_mun))
df_mun['COD_IBGE'] = df_mun['Ibge_6'].astype(str)

# Merges (PROCVs)
df_final = pd.merge(df_mun[['COD_IBGE', 'Nome_Municipio', 'CRS']], resumo_sinan[['COD_IBGE', 'Qtd_SINAN_Notificador']], on='COD_IBGE', how='left')
df_final = pd.merge(df_final, resumo_gal_solic[['COD_IBGE', 'Qtd_GAL_Solicitante']], on='COD_IBGE', how='left')
df_final = pd.merge(df_final, resumo_gal_resid[['COD_IBGE', 'Qtd_GAL_Residencia']], on='COD_IBGE', how='left')

df_final = df_final.fillna(0)

cols_contagem = ['Qtd_SINAN_Notificador', 'Qtd_GAL_Solicitante', 'Qtd_GAL_Residencia']
df_final[cols_contagem] = df_final[cols_contagem].astype(int)

# Lógica de Status (Prioridade da Chefe: Alerta no GAL sem SINAN)
def definir_status(row):
    if row['Qtd_GAL_Solicitante'] > 0 and row['Qtd_SINAN_Notificador'] == 0:
        return "🚨 ALERTA: GAL sem SINAN"
    elif row['Qtd_SINAN_Notificador'] > 0 and row['Qtd_GAL_Solicitante'] == 0:
        return "❓ SINAN sem GAL"
    elif row['Qtd_SINAN_Notificador'] > 0:
        return "✅ OK"
    return "Sem atividade"

df_final['STATUS_MONITORAMENTO'] = df_final.apply(definir_status, axis=1)

# Exportação para o CEVS
caminho_saida = os.path.join(PASTA_OUTPUT, f'Monitoramento_Dengue_{DATA_FIM}.xlsx')
df_final.to_excel(caminho_saida, index=False)

# --- ANONIMIZAÇÃO PARA O GITHUB ---
# Salva uma versão sem dados sensíveis na pasta data_demo
df_demo = df_final.copy()
df_demo.to_csv(os.path.join(PASTA_DEMO, 'resultado_monitoramento_demo.csv'), index=False)

print(f"🚀 Relatório gerado em: {caminho_saida}")
print(f"🔒 Dados anonimizados salvos em: {PASTA_DEMO}")

🚀 Relatório gerado em: ../02_Saida\Monitoramento_Dengue_2026-01-31.xlsx
🔒 Dados anonimizados salvos em: ../data_demo
